In [2]:
from ts_vae.data_processors.ts_gen_processor import TSGenDataset
from torch_geometric.data import DataLoader
import numpy as np
#from ts_vae.utils import remove_files
#remove_files() 
rxns = TSGenDataset(r'data')

In [3]:
tt_split = 0.8
num_rxns = len(rxns)
num_train = int(np.floor(tt_split * num_rxns))
batch_size = 4
#to_follow = ['edge_index_r', 'edge_index_ts', 'edge_index_p', 'edge_attr_r', 'edge_attr_ts', 'edge_attr_p'
#                'pos_r', 'pos_ts', 'pos_p', 'x_r', 'x_ts', 'x_p']
train_loader = DataLoader(rxns[: num_train], batch_size = batch_size)
test_loader = DataLoader(rxns[num_train: ], batch_size = batch_size)

In [48]:
# rxns[0]
batch = next(iter(train_loader))
test = batch.edge_attr

In [10]:
import torch
n = 10
a = torch.randn(2, n, n)
a = a * (1 - torch.eye(n)) # remove self loops
a, torch.square(a)

(tensor([[[-0.000,  1.336, -0.589,  0.743, -0.246, -0.781,  1.144,  0.605,
           -0.272,  1.757],
          [ 0.416,  0.000, -0.413, -1.241,  0.600,  0.053,  0.644, -0.614,
            0.296, -1.179],
          [ 0.761, -0.984,  0.000, -0.041, -1.536, -1.153,  0.148,  2.648,
           -0.994, -1.507],
          [ 0.078, -1.251, -0.024,  0.000, -0.948,  0.481,  0.303,  1.155,
            0.323, -0.738],
          [-2.637,  0.909, -0.804,  0.468, -0.000, -0.677,  0.869,  0.920,
            0.361,  0.355],
          [-0.087, -0.143, -0.773, -0.717, -0.036, -0.000,  1.621,  1.006,
           -0.588, -0.413],
          [ 0.266,  1.646,  0.426, -1.019,  0.105,  0.526,  0.000,  0.727,
            1.849,  1.226],
          [-0.065,  0.545, -0.059,  2.357,  0.246,  0.317,  1.227,  0.000,
           -0.444, -0.959],
          [-1.596, -0.111, -1.336, -0.953, -1.064,  0.287,  1.823, -1.030,
            0.000, -1.419],
          [-1.528, -0.183, -0.770, -1.218,  1.358, -1.190,  1.362, -1.277

In [1]:
from experiments.building_on_mit.meta_eval.meta_eval import ablation_experiment
from ts_vae.utils import remove_files
remove_files()
# have to use batch_size = 1 right now
train_log, test_log = ablation_experiment(0.8, 5, 2, 2)

Files removed.
Processing...


  1%|▏         | 100/7581 [00:00<00:24, 309.64it/s]


Done!
Starting ablation experiment...
===== Training epoch 001 complete with loss: 152.5986 ====
===== Training epoch 002 complete with loss: 40.7953 ====
===== Testing epoch 002 complete with loss: 52.3740 ====
Completed ablation experiment, use the experiment log to print results.


In [19]:
# dir(train_log)
train_log.epoch_ae_results[0]['D_init'][0].shape

torch.Size([21, 21])